<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">

# Desafío 2

# En desarrollo


## Alumno
Denardi, Fabricio

## Cohorte
15-2024

# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [21]:
import pandas as pd
import numpy as np
import os

import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow

import multiprocessing
from gensim.models import Word2Vec

### Datos
Utilizaremos como dataset canciones de bandas de habla inglesa.

In [7]:
def get_recipes_files(folder_paths):
    files = []
    for folder_path in folder_paths:
        files.extend([os.path.join(folder_path, file) for file in os.listdir(folder_path) if file.endswith('.txt') or file.endswith('.md')])
    return files

In [8]:
folder_paths = ['recipes_1', 'recipes_2']
all_files = get_recipes_files(folder_paths)
print("All files:")
print(all_files)

All files:
['recipes_1/agar.txt', 'recipes_1/almonds.txt', 'recipes_1/fava.txt', 'recipes_1/squares.txt', 'recipes_1/recipes2.txt', 'recipes_1/brownies.txt', 'recipes_1/pannacotta.txt', 'recipes_1/bruschetta.txt', 'recipes_1/sweet_potato_pie.txt', 'recipes_1/sorbet.txt', 'recipes_1/cauli.txt', 'recipes_1/tart.txt', 'recipes_1/cornbread.txt', 'recipes_2/crispy-beef-with-egg-fried-rice.md', 'recipes_2/classic-duck-breast.md', 'recipes_2/crispy-sesame-chicken.md', 'recipes_2/tomato-pasta.md', 'recipes_2/beef-stroganoff.md', 'recipes_2/carrot-cake.md', 'recipes_2/cacio-e-peppe.md', 'recipes_2/b-chamel-sauce.md', 'recipes_2/pizza-sauce.md', 'recipes_2/tagliatelle-with-broccoli-cauliflower-and-blue-cheese.md', 'recipes_2/pizza-dogs.md', 'recipes_2/meatballs.md', 'recipes_2/chicken-balmoral.md', 'recipes_2/chicken-broth-soup.md', 'recipes_2/thai-green-curry.md', 'recipes_2/blueberry-and-raspberry-muffins.md', 'recipes_2/prawn-pad-thai.md', 'recipes_2/toad-in-the-hole.md', 'recipes_2/garlic-do

In [14]:
# El encoding latin-1 es necesario para leer los archivos de texto. Si no se especifica, se produce un error de UnicodeDecodeError:
# UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe9 in position 2: invalid continuation byte
df = pd.DataFrame({'recipe': [open(file, encoding='latin-1').read() for file in all_files]})


In [18]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 82


In [129]:
df

,recipe
0,"AGAR-AGAR (KANTEN, CHINESE GELATIN)\n\nAsians..."
1,"BLANCHING NUTS: In the case of nuts, especiall..."
2,(fava beans)\n\ntreat gently when young and fr...
3,"These are kid tested, mother approved. From Co..."
4,\nless.\n\n1 cup dried split peas 1/2 teaspoon...
...,...
77,---\ntitle: Beef and Guinness stew\ndate: 2021...
78,---\ntitle: Margherita pizza\ndate: 2021-07-18...
79,---\ntitle: Whisky Haggis sauce\ndate: 2023-01...
80,---\ntitle: Bosnian Stuffed Peppers (Punjene P...


### 1 - Preprocesamiento

In [130]:
from tensorflow.keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row.iloc[0]))

    # El iloc reemplazó a la siguiente funcionalidad deprecated: row[0]


In [131]:
# Demos un vistazo
sentence_tokens[:5]

[['agar',
  'agar',
  'kanten',
  'chinese',
  'gelatin',
  'asians',
  'are',
  'fond',
  'of',
  'molded',
  'jellies',
  'some',
  'from',
  'fruits',
  'and',
  'nut',
  'milks',
  'but',
  'other',
  'made',
  'with',
  'sweetened',
  'beans',
  'and',
  'even',
  'corn',
  'but',
  'these',
  'are',
  'gelled',
  'with',
  'a',
  'seaweed',
  'derived',
  'gelatin',
  'called',
  'agar',
  'agar',
  'kanten',
  'in',
  'japanese',
  'rather',
  'than',
  'the',
  'gelatin',
  'derived',
  'from',
  'animal',
  'hooves',
  'and',
  'skins',
  'that',
  'is',
  'commonly',
  'used',
  'in',
  'western',
  'cooking',
  'for',
  'this',
  'reason',
  'agar',
  'it',
  'is',
  'generally',
  'referred',
  'to',
  'this',
  'way',
  'today',
  'has',
  'been',
  'adopted',
  'by',
  'western',
  'vegetarians',
  'and',
  'it',
  'is',
  'easily',
  'available',
  'in',
  'health',
  'food',
  'stores',
  'in',
  'bar',
  'flake',
  'and',
  'powder',
  'form',
  'a',
  'further',
  'ad

### 2 - Crear los vectores (word2vec)

In [26]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

In [336]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=10,       # dimensionalidad de los vectores 
                     negative=5,    # cantidad de negative samples... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [337]:
# Obtener el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [338]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 82


In [339]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.index_to_key))

Cantidad de words distintas en el corpus: 987


### 3 - Entrenar embeddings

In [340]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=200,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 267780.6875
Loss after epoch 1: 197914.9375
Loss after epoch 2: 184129.0
Loss after epoch 3: 174266.5
Loss after epoch 4: 167338.0
Loss after epoch 5: 162319.75
Loss after epoch 6: 160305.875
Loss after epoch 7: 156771.5
Loss after epoch 8: 154488.5
Loss after epoch 9: 153533.5
Loss after epoch 10: 152369.0
Loss after epoch 11: 152009.875
Loss after epoch 12: 147133.125
Loss after epoch 13: 145764.0
Loss after epoch 14: 145404.25
Loss after epoch 15: 146441.25
Loss after epoch 16: 145260.75
Loss after epoch 17: 143960.25
Loss after epoch 18: 143771.25
Loss after epoch 19: 144101.75
Loss after epoch 20: 144041.25
Loss after epoch 21: 143761.25
Loss after epoch 22: 143537.25
Loss after epoch 23: 145003.0
Loss after epoch 24: 143310.75
Loss after epoch 25: 143570.0
Loss after epoch 26: 137740.25
Loss after epoch 27: 131599.5
Loss after epoch 28: 131793.0
Loss after epoch 29: 131559.0
Loss after epoch 30: 131209.5
Loss after epoch 31: 131063.0
Loss after epoch 32: 13227

(5007888, 7683200)

### 4 - Ensayar

In [341]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["chocolate"], topn=10)

[('chips', 0.9116759300231934),
 ('spread', 0.8981676697731018),
 ('banana', 0.8757429718971252),
 ('caramel', 0.8724296689033508),
 ('crumbs', 0.8593956232070923),
 ('corn', 0.8590195775032043),
 ('raisins', 0.8500745892524719),
 ('unsweetened', 0.8397192358970642),
 ('even', 0.8209343552589417),
 ('cocktail', 0.8151442408561707)]

#### Comentarios
En esta primer palabra elegida para analizar, vemos claramente la eficacia del modelo. Choclate chips es una preparación de excelencia que encontraremos en cualquier set de recetas de pastelería.

Luego vemos, diferentes elemmentos que suelen combinarse con el chocolate, como la banana, el caramelo. Y en otras el tipo de chocolate, como el amargo (unsweetened).  También encontramos la relación con preparaciones menos convencionales, como el cocktail, el maiz (corn).

Veremos ahora otros análisis, pero anticipo un buen rendimiento del modelo.

In [342]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["pepper"], topn=10)

[('35', 0.21099022030830383),
 ('55', 0.20191547274589539),
 ('these', 0.1771249622106552),
 ('06', 0.17623025178909302),
 ('cutting', 0.1608394831418991),
 ('cornbread', 0.16011202335357666),
 ('16', 0.14946360886096954),
 ('33', 0.1424999237060547),
 ('times', 0.13918881118297577),
 ('brownies', 0.12367191165685654)]

#### Comentarios
Muchas de estos términos con similaridad más ortogonal, son núneros, con lo cual tienen que ver con el corpus elegido. Lo interesante es que brownies o pan de maiz, son algo que dificilmente lleve pimienta, y uno no corta (cutting) granos de pimienta, a lo sumo los muele, pero no con cuchillo.

In [343]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["carrot"], topn=10)

[('cake', 0.8617249131202698),
 ('soup', 0.8596341013908386),
 ('bosnian', 0.8545598387718201),
 ('mums', 0.8475525379180908),
 ('tart', 0.8469358086585999),
 ('fudge', 0.8249421715736389),
 ('classic', 0.818874180316925),
 ('pie', 0.8183954358100891),
 ('popular', 0.8094839453697205),
 ('jas', 0.8071134686470032)]

#### Comentarios
En este caso, vemos preparaciones realizadas con zanahoria, com por ejemplo cake/pie/tart (torta), soup (sopa), gudge (ganache, que es una especie de glaseado), el resto las asocio al contexto del set, ya que no están estrechamente relacionadas con zanahora.

Aclaración imporante: cuando digo que está relacionado con el contexto, me refiero a que si agrego otros dataset de diferentes origenes, posiblemente no obtenga estas palabras, quiero decir que es poco probable que aparezcan en otras recetas. En cambio carrot cake, es algo que en cualquier libro de pastelería veremos.

In [344]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["oven"], topn=5)

[('350', 0.9028052091598511),
 ('preheat', 0.9018155336380005),
 ('degrees', 0.8800850510597229),
 ('middle', 0.8728640675544739),
 ('line', 0.872209906578064)]

### Comentarios
Creo que acá es donde vemos las mejores analogías. 350 son los grados en Farenheit, que equivalen a 180 centígrados, temperatura más usada en el horneado de alimentos.

Luego vemos palabras como preheat, que significa precalentar el horno, usado usualmente en el preparado de tortas, budines, panes. Degrees que es la unidad de medida de temperatura (Grados). Middle, que es precalentar a temperatura media (muy usado en pastelería).

El match con line lo asocio exclusivamente con el corpus elegido, ya que no es un término habitual en la gastronomía.

In [345]:
# Ensayar con una palabra que no está en el vocabulario:
w2v_model.wv.most_similar(negative=["knife"], topn=5)

[('honey', 0.2076164335012436),
 ('japanese', 0.16702862083911896),
 ('png', 0.13680334389209747),
 ('reference', 0.13050629198551178),
 ('katsu', 0.1304164081811905)]

#### Comentarios
Claramente la primer relación negativa la vemos en que la miel, en el contexto de preparados de recetas, no se suele servir con cuchillos. El resto de las relaciones, son palabras que nada tiene que ver con el contexto gastronómico (ej: png es un formato de imagen).

In [346]:
# el método `get_vector` permite obtener los vectores:
vector_chocolate = w2v_model.wv.get_vector("chocolate")
print(vector_chocolate)

[ 0.7023125   0.2815829  -0.48930097 -0.40254492 -0.23666996  0.76835686
  0.82392955  0.6080292   0.4156515  -1.5616496 ]


In [347]:
# el método `most_similar` también permite comparar a partir de vectores
w2v_model.wv.most_similar(vector_chocolate)

[('chocolate', 1.0),
 ('chips', 0.9116759300231934),
 ('spread', 0.898167610168457),
 ('banana', 0.8757429718971252),
 ('caramel', 0.8724296689033508),
 ('crumbs', 0.8593956232070923),
 ('corn', 0.8590195775032043),
 ('raisins', 0.8500745892524719),
 ('unsweetened', 0.8397192358970642),
 ('even', 0.8209343552589417)]

#### Comentarios
Ya analizamos estos valores anteriormente.

In [348]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["rice"], topn=10)

[('lamb', 0.9363440871238708),
 ('saffron', 0.8996533155441284),
 ('pasta', 0.888781726360321),
 ('ratatouille', 0.8871985077857971),
 ('meat', 0.8820939064025879),
 ('sticky', 0.8492037057876587),
 ('yoghurt', 0.847372829914093),
 ('bã©chamel', 0.84041827917099),
 ('tomato', 0.8320508599281311),
 ('steak', 0.8255939483642578)]

#### Comentarios
Dado que el source contiene muchas recetas indias, el primer match se da con el cordero, que es un plato típico de ese país. Adicionalmente, vemos que las siguientes coincidencia la obtenemos o con sustitutos, uso del arroz comom acompañamiento del plato principal, o combinación de ingredientes junto con el arroz.

In [350]:
test_vector = w2v_model.wv["vinegar"].shape
test_vector

(10,)

### Test de analogía por resta de vectores
Simplemente para ampliar mi análisis, decidí construir un test de analogía como tal, restando vestores y buscando la similaridad del coseno entre ambos.

In [349]:
from sklearn.metrics.pairwise import cosine_similarity

def test_analogy(model: Word2Vec, word1: str, word2: str, word3: str, word4: str) -> float:
    try:
        vector1 = model.wv[word1]
        vector2 = model.wv[word2]
        vector3 = model.wv[word3]
        vector4 = model.wv[word4]

        result = cosine_similarity([vector2 - vector1], [vector4 - vector3])[0][0]
        return result
    except KeyError:
        return 0
    


In [360]:
analogy_list = [
    ("banana", "fruit", "carrot", "vegetable"),
    ("teaspoon", "salt", "cup", "sugar"),
    ("salt", "pepper", "sugar", "chocolate"),
    ("flour", "bake", "water", "boil"),
    ("bread", "butter", "pasta", "sauce"),
    ("onion", "soup", "garlic", "pasta"),
]

for analogy in analogy_list:
    result = test_analogy(w2v_model, analogy[0], analogy[1], analogy[2], analogy[3])

    print(f'Analogía: "{analogy[0]}" es a "{analogy[1]}" como "{analogy[2]}" es a "{analogy[3]}"')
    print(f"Similaridad: {result:.2f}")
    print("-----------------------------")

Analogía: "banana" es a "fruit" como "carrot" es a "vegetable"
Similaridad: 0.41
-----------------------------
Analogía: "teaspoon" es a "salt" como "cup" es a "sugar"
Similaridad: 0.15
-----------------------------
Analogía: "salt" es a "pepper" como "sugar" es a "chocolate"
Similaridad: 0.43
-----------------------------
Analogía: "flour" es a "bake" como "water" es a "boil"
Similaridad: 0.50
-----------------------------
Analogía: "bread" es a "butter" como "pasta" es a "sauce"
Similaridad: 0.58
-----------------------------
Analogía: "onion" es a "soup" como "garlic" es a "pasta"
Similaridad: 0.89
-----------------------------


#### Conclusión del punto
He encontrado algunas analogías interesantes. Ahora bien, como era de esperarse, no logré detectar una analogía perfecta. Esto se debe a que dependemos exclusivamente del corpus elegido. Si bien son todas recetas, las mismas perteneces a diferentes orígenes de datos y además son de diferentes paises, culturas, sabores, etc, lo que hace que tenga relaciones muy heterogénas.

### 5 - Visualizar agrupación de vectores

In [352]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [353]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show() # esto para plotly en colab

In [354]:
# Graficar los embedddings en 3D

vecs, labels = reduce_dimensions(w2v_model,3)

fig = px.scatter_3d(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], z=vecs[:MAX_WORDS,2],text=labels[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show() # esto para plotly en colab

In [355]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors.tsv", vectors, delimiter="\t")

with open("labels.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

In [356]:
# También se pueden guardar los vectores y labels como tsv para graficar en
# http://projector.tensorflow.org/


vectors = np.asarray(w2v_model.wv.vectors)
labels = list(w2v_model.wv.index_to_key)

np.savetxt("vectors_recipes.tsv", vectors, delimiter="\t")

with open("label_recipes.tsv", "w") as fp:
    for item in labels:
        fp.write("%s\n" % item)

# Conclusiones finales
1. La performance del modelo entrenado es muy buena. A pesar de las limitaciones del mismo, pude encontrar relaciones muy interesantes, la mayoría de esas esperadas, aunque otras que dependen exclusivamente del contexto de las recetas elegidas, que requeriran de empliar el dataset para evaluar si siguen o no siendo relevamentes, en cuyo caso, habría que ver el origen de esta relación.

2. Para el análisis de este y cualquier problema, es necesario contar con conocimientos del tema elegido, al menos para evaluar la efectividad del mismo. Mi hobbie en gastronomía, me permitió realizar un buen análisis del mismo.

3. Vemos en el gráfico del punto anterior que la reducción de dimensiones, tuvo bastante éxito. Veo, sin embargo, que palabras genéricas o no asociadas a la gastronomía, tienen cercanía con otros término vinculados a dicha temática, dificultando, a mi criterio, un análisis más limpio. No quise seguir sumando al hiperparámetro de cantidad mínima de apariciones, porque me eliminaba términos muy interesante y que, si bien no aparecen en todas las recetas, están estrechamente relacionadas con el tema de interés, como por ejemplo la canela.

